# ĐỒ ÁN CUỐI KỲ - NHẬP MÔN KHOA HỌC DỮ LIỆU
STT nhóm: 17

Thành viên:
- 18120066 - Bùi Đoàn Hữu Nhân
- 18120097 - Đinh Hữu Phúc Trung

##  Đề tài: Dự đoán giá xe ô tô

---

# II - Tiền xử lý và mô hình hóa
Trong file notebook này ta sẽ thực hiện tiền xử lý và mô hình hóa dữ liệu đã được thu thập sẵn như đã trình bày trong file `DACK-ThuThapDuLieu.ipynb`

---

## Import

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.neural_network import MLPClassifier
from sklearn import set_config
set_config(display='diagram')

---

## Khám phá dữ liệu (đủ để đặt câu hỏi)

In [ ]:
data_df = pd.read_csv('cars_data.csv', index_col=0) # Cho cột index là cột 0
data_df.head()

### Dữ liệu có bao nhiêu dòng và bao nhiêu cột?

In [ ]:
data_df.shape

### Mỗi dòng có ý nghĩa gì? Có vấn đề các dòng có ý nghĩa khác nhau không?

Quan sát sơ bộ dữ liệu ta thấy mỗi dòng chứa thông tin của một xe ôtô, và không có vấn đề các dòng có ý nghĩa khác nhau.

### Dữ liệu có các dòng bị lặp không?

In [ ]:
data_df.index.duplicated().sum()

### Mỗi cột có ý nghĩa gì?

Ý nghĩa của mỗi cột như sau:

    Name                  Tên xe
    Brand                 Hãng xe
    Price                 Giá xe
    Body                  Loại thân xe
    Transmission          Số cấp của hộp số
    Number Of Seats       Số chỗ ngồi
    Segment               Loại kích cỡ xe
    Introduction          Năm sản xuất
    Drive                 Hệ thống dẫn động
    Drive System          Loại động cơ
    Fuel                  Loại nhiên liệu
    Cylinder Capacity     Dung tích xilanh
    Max Power Hp          Công suất tối đa đơn vị là mã lực
    Max Torque            Momen xoắn cực đại
    Fuel System           Hệ thống nhiên liệu
    Valve Actuation       Kiểu kích hoạt van
    Turbo                 Bộ tăng áp
    Fuel Tank             Dung tích bình nhiên liệu
    Top Speed             Tốc độ tối đa
    Energy Label          Nhãn năng lượng
    Front Stabilizer      Bộ ổn định phía trước
    Rear Stabilizer       Bộ ổn định phía sau
    Num_doors             Số cửa
    Dt_Transmission       Loại hộp số

---

## Đưa ra câu hỏi cần trả lời

**Câu hỏi:**

*Output - giá ôtô -* được tính từ *input - các đặc trưng, bộ phận của ôtô -* theo công thức nào?

**Lợi ích:**

Việc tìm ra câu trả lời cho câu hỏi này sẽ có lợi ích là:
- Về phía người bán: giúp định giá được xe ôtô dựa vào cấu tạo của xe.
- Về phía người mua: giúp người mua nắm được giá theo thị trường của loại xe đó khi biết các đặc trưng của xe.

---

## Khám phá dữ liệu (để biết cách tách các tập)

In [ ]:
# Cột output hiện có kiểu dữ liệu gì?
data_df['Price'].dtype

In [ ]:
# Cột output có giá trị thiếu không?
data_df['Price'].isna().sum()

Như vậy là không có vấn đề gì cần xử lí ở đây cả.

---

## Tiền xử lý (tách các tập)

Bây giờ ta sẽ thực hiện bước tiền xử lý là tách tập validation và tập kiểm tra ra.

In [ ]:
# Tách X và y
y_sr = data_df['Price']
X_df = data_df.drop(['Price'], axis=1)

In [ ]:
# Tách tập kiểm tra
X_df, test_X_df, y_sr, test_y_sr = train_test_split(X_df, y_sr, test_size=0.2, random_state=0)

In [ ]:
# Tách tập validation
train_X_df, val_X_df, train_y_sr, val_y_sr = train_test_split(X_df, y_sr, test_size=0.25, random_state=0)

In [ ]:
train_X_df.shape

In [ ]:
train_y_sr.shape

In [ ]:
val_X_df.shape

In [ ]:
val_y_sr.shape

In [ ]:
test_X_df.shape

In [ ]:
test_y_sr.shape

In [ ]:
train_X_df.head().index

---

## Khám phá dữ liệu (tập huấn luyện)

Sau khi đã tách ra các tập thì ta có thể thoải mái khám phá trên tập huấn luyện mà không lo sẽ làm kết quả trên tập validation và tập kiểm tra bị mất đi sự khách quan.

### Mỗi cột input hiện đang có kiểu dữ liệu gì? Có cột nào có kiểu dữ liệu chưa phù hợp để có thể xử lý tiếp không?

In [ ]:
train_X_df.dtypes

Có vẻ các cột đều có kiểu dữ liệu phù hợp. 

### Với mỗi cột input có kiểu dữ liệu dạng số, các giá trị được phân bố như thế nào?

Trong `train_X_df`, có 5/10 cột có dtype không phải là object:

In [ ]:
train_X_df.dtypes[train_X_df.dtypes != object]

In [ ]:
num_cols = ['Transmission', 'Number Of Seats', 'Introduction', 'Cylinder Capacity', 'Max Power Hp', 
            'Max Torque', 'Fuel Tank', 'Top Speed', 'Num_doors']
df = train_X_df[num_cols]
def missing_ratio(df):
    return (df.isna().mean() * 100).round(1)
def lower_quartile(df):
    return df.quantile(0.25).round(1)
def median(df):
    return df.quantile(0.5).round(1)
def upper_quartile(df):
    return df.quantile(0.75).round(1)
df.agg([missing_ratio, 'min', lower_quartile, median, upper_quartile, 'max'])

### Với mỗi cột input có kiểu dữ liệu không phải dạng số, các giá trị được phân bố như thế nào?

In [ ]:
pd.set_option('display.max_colwidth', 200) # Để nhìn rõ hơn
cat_cols = list(set(train_X_df.columns) - set(num_cols))
df = train_X_df[cat_cols]
def missing_ratio(df):
    return (df.isna().mean() * 100).round(1)
def num_values(df):
    return df.nunique()
def value_ratios(c):
    return dict((c.value_counts(normalize=True) * 100).round(1))
df.agg([missing_ratio, num_values, value_ratios])

---

## Tiền xử lý (tập huấn luyện)

Đầu tiên, ta sẽ xử lý một số cột như sau: 
- Bỏ cột "Name" và "Brand" vì có rất nhiều giá trị khác nhau.
- Bỏ cột "Energy Label" vì có quá nhiều giá trị thiếu.
- Ở cột "Fuel System" và "Dt_Transmission" ta sẽ lấy top các giá trị xuất hiện nhiều nhất.

*Cột "Segment" và cột "Body" cũng có nhiều giá trị, nhưng vì các giá trị có tỉ lệ xuất hiện không quá chênh lệch nên ta giữ nguyên*

In [ ]:
class ColAdderDropper(BaseEstimator, TransformerMixin):
    def __init__(self, num_top_fuel_sys=1, num_top_transmission=1):
        self.num_top_fuel_sys = num_top_fuel_sys
        self.num_top_transmission = num_top_transmission
    def fit(self, X_df, y=None):
        # Fuel System
        self.fuel_sys_counts_ = X_df['Fuel System'].value_counts()
        fuel_sys = list(self.fuel_sys_counts_.index)
        self.top_fuel_sys_ = fuel_sys[:max(1, min(self.num_top_fuel_sys, len(fuel_sys)))]
        # Transmission
        self.transmission_counts_ = X_df['Dt_Transmission'].value_counts()
        transmission = list(self.transmission_counts_.index)
        self.top_transmission_ = transmission[:max(1, min(self.num_top_transmission, len(transmission)))]
        return self
    def transform(self, X_df, y=None):
        new_df = X_df
        # Fuel System
        fuel_sys_list = list(new_df['Fuel System'])
        for i in range(len(fuel_sys_list)):
            if fuel_sys_list[i] not in self.top_fuel_sys_:
                fuel_sys_list[i] = 'others'
        new_df['Fuel System'] = fuel_sys_list
        # Transmission
        transmission_list = list(new_df['Dt_Transmission'])
        for i in range(len(transmission_list)):
            if transmission_list[i] not in self.top_transmission_:
                transmission_list[i] = 'others'
        new_df['Dt_Transmission'] = transmission_list
        new = new_df.drop(['Name', 'Brand', 'Energy Label'], axis=1)
        return new_df

In [ ]:
col_adderdropper = ColAdderDropper(num_top_fuel_sys=1, num_top_transmission=1)
col_adderdropper.fit(train_X_df)
print(col_adderdropper.top_fuel_sys_)
print()
print(col_adderdropper.fuel_sys_counts_)
print()
print(col_adderdropper.top_transmission_)
print()
print(col_adderdropper.transmission_counts_)

In [ ]:
fewer_cols_train_X_df = col_adderdropper.transform(train_X_df)
fewer_cols_train_X_df.head()

Các bước tiền xử lý tiếp theo như sau:
- Với các cột dạng số, ta sẽ điền giá trị thiếu bằng giá trị mean của cột <font color=gray>(gợi ý: dùng `SimpleImputer` trong Sklearn)</font>. Với *tất cả* các cột dạng số trong tập huấn luyện, ta đều cần tính mean, vì ta không biết được cột nào sẽ bị thiếu giá trị khi dự đoán với các véc-tơ input mới. 
- Với các cột không phải dạng số và không có thứ tự:
    - Ta sẽ điền giá trị thiếu bằng giá trị mode (giá trị xuất hiện nhiều nhất) của cột <font color=gray>(gợi ý: dùng `SimpleImputer` trong Sklearn)</font>. Với *tất cả* các cột không có dạng số và không có thứ tự, ta đều cần tính mode, vì ta không biết được cột nào sẽ bị thiếu giá trị khi dự đoán với các véc-tơ input mới.
    - Sau đó, ta sẽ chuyển sang dạng số bằng phương pháp mã hóa one-hot <font color=gray>(gợi ý: dùng `OneHotEncoder` trong Sklearn, để ý tham số `handle_unknown` vì khi dự đoán với các véc-tơ input mới ...)</font>.

- Cuối cùng, khi tất cả các cột đã được điền giá trị thiếu và đã có dạng số, ta sẽ tiến hành chuẩn hóa bằng cách trừ đi mean và chia cho độ lệch chuẩn của cột để giúp cho các thuật toán cực tiểu hóa như Gradient Descent, LBFGS, ... hội tụ nhanh hơn <font color=gray>(gợi ý: dùng `StandardScaler` trong Sklearn)</font>.

Nhiệm vụ của bạn là tạo ra một pipeline, đặt tên là `preprocess_pipeline`, bao gồm: các bước cài ở class `ColAdderDropper`, và tất cả các bước ở đây. Sau khi tạo ra được pipeline này rồi, bạn sẽ gọi phương thức `fit_transform` với đầu vào là `train_X_df` để tính các giá trị từ tập huấn luyện và đồng thời tiền xử lý `train_X_df`; kết quả trả về sẽ là `train_X_df` sau khi đã tiền xử lý, là một mảng Numpy, bạn đặt tên là `preprocessed_train_X`.

In [ ]:
# Tạo pipepline xử lí
num_cols = ['Transmission', 'Number Of Seats', 'Introduction', 'Cylinder Capacity', 'Max Power Hp', 
            'Max Torque', 'Fuel Tank', 'Top Speed', 'Num_doors']
unorder_cate_cols = list(set(train_X_df.columns) - set(nume_cols))
# YOUR CODE HERE
col_trans = make_column_transformer((SimpleImputer(missing_values=np.nan, strategy='mean'), nume_cols), 
                                    (make_pipeline(SimpleImputer(missing_values=np.nan, strategy='most_frequent'), 
                                                   OneHotEncoder(handle_unknown="ignore")), unorder_cate_cols))
# Preprocess pipeline
preprocess_pipeline = make_pipeline(ColAdderDropper(num_top_fuel_sys=3, num_top_transmission=2), 
                                    col_trans, StandardScaler(with_mean=False))

preprocessed_train_X = preprocess_pipeline.fit_transform(train_X_df)

In [ ]:
preprocess_pipeline

## Tiền xử lý (tập validation) (1.5đ)

Một khi đã có `preprocess_pipeline` với các giá trị đã được tính từ tập huấn luyện, ta có thể dễ dàng dùng phương thức `transform` để tiền xử lý cho các véc-tơ input mới trong tập validation và tập kiểm tra. Dưới đây, bạn sẽ làm như vậy để tiền xử lý cho `val_X_df` và lưu kết quả vào `preprocessed_val_X`.

In [ ]:
preprocessed_val_X = preprocess_pipeline.transform(val_X_df)

---

## Tiền xử lý + mô hình hóa

### Tìm mô hình tốt nhất 

Ta sẽ sử dụng mô hình Neural Net để phân lớp. Bạn sẽ tạo ra một pipeline từ đầu đến cuối bao gồm: các bước tiền xử lý ở trên + Neural Net (với các siêu tham số `hidden_layer_sizes=(30), activation='relu', solver='adam', random_state=0, max_iter=2500`). Đặt tên cho pipeline này là `full_pipeline`. 

In [ ]:
# Tạo full pipeline
# Train mô hình và chọn siêu tham số
# Tạo full pipeline
nume_cols = ['Number Of Seats', 'Introduction', 'Cylinder Capacity', 'Max Power Hp', 'Max Torque', 'Fuel Tank', 'Top Speed', 'Combined Consumption']
unorder_cate_cols = list(set(train_X_df.columns) - set(nume_cols))
# YOUR CODE HERE
cd = ColAdderDropper(num_top_fuel_sys=3, num_top_transmission=2)
full_pipeline = make_pipeline(cd, col_trans, StandardScaler(with_mean=False), 
                              MLPRegressor(hidden_layer_sizes=(30), activation='relu', solver='adam', random_state=0, max_iter=2500))

# Thử nghiệm với các giá trị khác nhau của các siêu tham số
# và chọn ra các giá trị tốt nhất
train_errs = []
val_errs = []
alphas = [1000]
num_top_titles_s = [1, 3, 5, 7, 9, 11]
best_val_err = float('inf'); best_alpha = None; best_num_top_titles = None
for alpha in alphas:
    # YOUR CODE HERE
    full_pipeline.set_params(mlpregressor__alpha=alpha)
    full_pipeline.fit(train_X_df, train_y_sr)

    train_err = (1 - full_pipeline.score(train_X_df, train_y_sr))*100
    val_err = (1 - full_pipeline.score(val_X_df, val_y_sr))*100
    #val_err = (1 - compute_rr(val_y_sr, full_pipeline.predict(val_X_df), baseline_preds))*100
    train_errs.append(train_err)
    val_errs.append(val_err)

    if val_err < best_val_err:
        best_val_err = val_err
        best_alpha = alpha
'Finish!'

In [ ]:
# TEST
full_pipeline

Cuối cùng, ta sẽ huấn luyện lại `full_pipeline` trên `X_df` và `y_sr` (tập huấn luyện + tập validation) với `best_alpha` tìm được ở trên để ra được mô hình cụ thể cuối cùng.

In [ ]:
# YOUR CODE HERE
full_pipeline.set_params(mlpclassifier__alpha=best_alpha)
full_pipeline.fit(X_df, y_sr)

### Đánh giá mô hình tìm được 

In [ ]:
full_pipeline.score(test_X_df, test_y_sr)*100